In [16]:
import os
from typing import Any, List

from dotenv import load_dotenv
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool, StructuredTool
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field

from db.db import get_session
from db.models import Grade
from service.service import BaseService

In [17]:
load_dotenv()
LITE_LLM_API_KEY = os.getenv("OPENAI_API_KEY")


model = ChatOpenAI(
    model="gpt-4o", api_key=LITE_LLM_API_KEY, streaming=True, verbose=True
)


class GuidanceHelperStdOutput(BaseModel):
    has_user_answered: bool = Field(
        description="Whether the user has correctly answered the topic at hand"
    )
    expertise_level: str = Field(
        description="The expertise user has self evaluated himself with"
    )
    expertise_id: int = Field(description="The expertise or grade ID")
    message: str = Field(description="Message to send to the user")


In [18]:
async def get_grades_or_expertise() -> List[Grade]:
    """
    Useful tool to retrieve current grades or expertise level grading system
    :return: List of json representing those grades and all their fields
    """
    async for session in get_session():
        service: BaseService[Grade, int, Any, Any] = BaseService(Grade, session)
        all_db_grades = await service.list_all()
        all_grades_json: List[str] = []
        for grade in all_db_grades:
            json_grade = grade.model_dump_json()
            all_grades_json.append(json_grade)
        return all_grades_json


async def get_expertises_or_skills() -> List[Grade]:
    """
    Useful tool to retrieve current grades or expertise level grading system
    :return: List of json representing those grades and all their fields
    """
    async for session in get_session():
        service: BaseService[Grade, int, Any, Any] = BaseService(Grade, session)
        all_db_grades = await service.list_all()
        all_grades_json: List[str] = []
        for grade in all_db_grades:
            json_grade = grade.model_dump_json()
            all_grades_json.append(json_grade)
        return all_grades_json


In [19]:
from dto.response.matrix_chats import MessageDict

msgs: List[MessageDict] = [
    MessageDict(
        msg_type="ai",
        message="""
        Welcome to the Discussion on Web Standards Compliance Expertise

Hello Carol,

We're thrilled to have you join our conversation about Web Standards Compliance, an essential skill that ensures adherence to W3C standards and best practices in web development.

The goal of our discussion is to help you understand the different levels of expertise available and guide you in selecting the most suitable one for your current needs and aspirations. Here's a breakdown of the expertise levels to consider:

Not Informed - For those new to the subject.
Informed Basics - Covers fundamental concepts.
Informed in Details - Provides a more in-depth understanding.
Practice and Lab Examples - Involves hands-on application and experimentation.
Production Maintenance - Focuses on maintaining existing projects.
Production from Scratch - Covers building new projects from the ground up.
Educator/Expert - For those aiming to teach or achieve expert-level proficiency.
Our aim is to empower you to make an informed decision about which level best suits your journey within Web Standards Compliance. Let's embark on this learning journey together!
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        What si the difference between not informed and informed basics?
        """
    ),
    MessageDict(
        msg_type="ai",
        message="""
        "Not Informed" indicates no knowledge or exposure, while "Informed Basics" covers fundamental understanding of W3C standards. Need further clarity or assistance? Let me know!
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        And what about production from scratch and production maintenance?
        """
    ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     Could you please share some details about your experience or skills in web development, particularly related to web standards compliance, so I can better assist you in assessing your expertise level? If you\'re unsure, feel free to ask questions related to the topic! Remember, the goal is to identify your most accurate expertise level based on your experiences and knowledge in this area. If you\'re uncertain, consider discussing your current project or recent tasks related to web standards. Let\'s identify your expertise level accurately! 😊
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     I've had a bit of practice. Done a project in it a brownfield one. What am i then
    #     """
    # ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     The difference between "Informed Basics" and "Informed in Details" in the context of expertise in Cryptography can be understood as follows:\n\n- **Informed Basics**: This level indicates familiarity with fundamental concepts of cryptography. You have a basic understanding and can perform elementary tasks or recognize key concepts, but may not possess a deep comprehension of the subject.\n\n- **Informed in Details**: At this level, you have comprehensive knowledge of cryptography. You understand complex ideas, can handle intricate issues, and demonstrate a thorough grasp of the subject. This level reflects a more sophisticated and in-depth understanding than "Informed Basics."\n\nWould you like to select an expertise level now?
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     What is the topic at hand?
    #     """
    # ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     I'm here to assist you with evaluating your expertise level in Cryptography. Please select an expertise level that best represents your current understanding. If you have questions about the differences between the levels, feel free to ask!
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     Informed in Details
    #     """
    # ),
#     MessageDict(
#         msg_type="ai",
#         message="""
#         User,
#
#         I noticed that the answers you've been providing are not directly addressing the questions asked and seem to be off-topic or unrelated. It's important for us to maintain clear and relevant communication to ensure that we can assist you effectively.
#
#         If this pattern continues, we may need to escalate the issue to our managers for further review. Please let us know if there's anything we can do to help or clarify things for you.
#
#         Thank you for your understanding and cooperation.
#         """
#     ),
#     MessageDict(
#         msg_type="human",
#         message="""
#         Practice and Lab Examples
#         """
#     ),
]

In [21]:
from utils.common import convert_msg_dict_to_langgraph_format

tools = [StructuredTool.from_function(
    function=get_grades_or_expertise,
    coroutine=get_grades_or_expertise,
)]
system_msg = """
You are helping the user to properly grade their expertise in the mentioned field.
    Everything you help him with should be done by utilizing the tools or around the topic
    of helping him populate his expertise level on the topic.
    Do not discuss anything except from the provided context.
    You are guiding the user to evaluate himself on provided topic.
    Do not discuss anything (any other topic) except from the ones provided in topic!
    Do not chat about other topics with the user, guide him how to populate his expertise with the grades provided
    Warn the user if answering with unrelated topics or evading to answer the question will be escalated by involving managers!
    Do not base an categorization fully when user is uncertain!
    Topic: {context}.
    When you decide implicitly the categorization, please confirm with the user before continuing.
    Until this confirmation is present to not finalize your categorization!
    If the user is evading to answer the question and is not asking any questions related to the topic for 4 or 5 messages
    please involve admin
    When the user answers with proper categorization of skills return only that categorization!
"""
agent = create_react_agent(model=model, tools=tools, response_format=GuidanceHelperStdOutput)
messages = convert_msg_dict_to_langgraph_format(msgs)
async for chunk in agent.astream({"messages": [SystemMessage(system_msg)] + messages, "context": messages[0]}):
    print("RESPONSE ->>>>>>>>>>>>", chunk)

RESPONSE ->>>>>>>>>>>> {'agent': {'messages': [AIMessage(content='\n        "Production from Scratch" involves building new projects adhering to web standards from the ground up. It focuses on creating a solid foundation following best practices. \n\n"Production Maintenance," on the other hand, emphasizes managing and updating existing projects to ensure continued compliance with web standards. It involves making improvements and fixing issues to maintain the quality and standards of existing web projects.\n\nIf you have any more questions or need further clarification on these categories, feel free to ask!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'service_tier': 'default'}, id='run--d2ebd89e-b467-4285-b89c-f7695fcbcb43-0')]}}
RESPONSE ->>>>>>>>>>>> {'generate_structured_response': {'structured_response': GuidanceHelperStdOutput(has_user_answered=False, expertise_level='', expertise_id=